# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f68181e1dc0>
├── 'a' --> tensor([[ 1.3664,  0.5328, -0.6955],
│                   [ 0.7239, -0.5734,  1.5344]])
└── 'x' --> <FastTreeValue 0x7f68181e1d30>
    └── 'c' --> tensor([[-0.1460,  1.6373, -0.0251, -1.8319],
                        [-0.7678, -1.0869, -1.2274, -1.3800],
                        [ 0.9530, -0.0147,  1.8584, -0.3584]])

In [4]:
t.a

tensor([[ 1.3664,  0.5328, -0.6955],
        [ 0.7239, -0.5734,  1.5344]])

In [5]:
%timeit t.a

62.9 ns ± 0.027 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f68181e1dc0>
├── 'a' --> tensor([[ 0.5175, -1.2652, -0.7014],
│                   [ 1.4347, -1.2194, -0.2773]])
└── 'x' --> <FastTreeValue 0x7f68181e1d30>
    └── 'c' --> tensor([[-0.1460,  1.6373, -0.0251, -1.8319],
                        [-0.7678, -1.0869, -1.2274, -1.3800],
                        [ 0.9530, -0.0147,  1.8584, -0.3584]])

In [7]:
%timeit t.a = new_value

59.4 ns ± 0.024 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3664,  0.5328, -0.6955],
               [ 0.7239, -0.5734,  1.5344]]),
    x: Batch(
           c: tensor([[-0.1460,  1.6373, -0.0251, -1.8319],
                      [-0.7678, -1.0869, -1.2274, -1.3800],
                      [ 0.9530, -0.0147,  1.8584, -0.3584]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3664,  0.5328, -0.6955],
        [ 0.7239, -0.5734,  1.5344]])

In [11]:
%timeit b.a

58.9 ns ± 0.0263 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.7239, -0.3620,  0.2066],
               [-1.1177,  0.6413, -1.2360]]),
    x: Batch(
           c: tensor([[-0.1460,  1.6373, -0.0251, -1.8319],
                      [-0.7678, -1.0869, -1.2274, -1.3800],
                      [ 0.9530, -0.0147,  1.8584, -0.3584]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.103 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 0.0575 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 31.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 451 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 401 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6772af0850>
├── 'a' --> tensor([[[ 1.3664,  0.5328, -0.6955],
│                    [ 0.7239, -0.5734,  1.5344]],
│           
│                   [[ 1.3664,  0.5328, -0.6955],
│                    [ 0.7239, -0.5734,  1.5344]],
│           
│                   [[ 1.3664,  0.5328, -0.6955],
│                    [ 0.7239, -0.5734,  1.5344]],
│           
│                   [[ 1.3664,  0.5328, -0.6955],
│                    [ 0.7239, -0.5734,  1.5344]],
│           
│                   [[ 1.3664,  0.5328, -0.6955],
│                    [ 0.7239, -0.5734,  1.5344]],
│           
│                   [[ 1.3664,  0.5328, -0.6955],
│                    [ 0.7239, -0.5734,  1.5344]],
│           
│                   [[ 1.3664,  0.5328, -0.6955],
│                    [ 0.7239, -0.5734,  1.5344]],
│           
│                   [[ 1.3664,  0.5328, -0.6955],
│                    [ 0.7239, -0.5734,  1.5344]]])
└── 'x' --> <FastTreeValue 0x7f6772af0e20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 75 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f676bed3f40>
├── 'a' --> tensor([[ 1.3664,  0.5328, -0.6955],
│                   [ 0.7239, -0.5734,  1.5344],
│                   [ 1.3664,  0.5328, -0.6955],
│                   [ 0.7239, -0.5734,  1.5344],
│                   [ 1.3664,  0.5328, -0.6955],
│                   [ 0.7239, -0.5734,  1.5344],
│                   [ 1.3664,  0.5328, -0.6955],
│                   [ 0.7239, -0.5734,  1.5344],
│                   [ 1.3664,  0.5328, -0.6955],
│                   [ 0.7239, -0.5734,  1.5344],
│                   [ 1.3664,  0.5328, -0.6955],
│                   [ 0.7239, -0.5734,  1.5344],
│                   [ 1.3664,  0.5328, -0.6955],
│                   [ 0.7239, -0.5734,  1.5344],
│                   [ 1.3664,  0.5328, -0.6955],
│                   [ 0.7239, -0.5734,  1.5344]])
└── 'x' --> <FastTreeValue 0x7f676bed3fd0>
    └── 'c' --> tensor([[-0.1460,  1.6373, -0.0251, -1.8319],
                        [-0.7678, -1.0869, -1.2274, -1.3800],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 41.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.6 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.3664,  0.5328, -0.6955],
                [ 0.7239, -0.5734,  1.5344]],
       
               [[ 1.3664,  0.5328, -0.6955],
                [ 0.7239, -0.5734,  1.5344]],
       
               [[ 1.3664,  0.5328, -0.6955],
                [ 0.7239, -0.5734,  1.5344]],
       
               [[ 1.3664,  0.5328, -0.6955],
                [ 0.7239, -0.5734,  1.5344]],
       
               [[ 1.3664,  0.5328, -0.6955],
                [ 0.7239, -0.5734,  1.5344]],
       
               [[ 1.3664,  0.5328, -0.6955],
                [ 0.7239, -0.5734,  1.5344]],
       
               [[ 1.3664,  0.5328, -0.6955],
                [ 0.7239, -0.5734,  1.5344]],
       
               [[ 1.3664,  0.5328, -0.6955],
                [ 0.7239, -0.5734,  1.5344]]]),
    x: Batch(
           c: tensor([[[-0.1460,  1.6373, -0.0251, -1.8319],
                       [-0.7678, -1.0869, -1.2274, -1.3800],
                       [ 0.9530, -0.0147,  1.8584, -0.3584]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.3664,  0.5328, -0.6955],
               [ 0.7239, -0.5734,  1.5344],
               [ 1.3664,  0.5328, -0.6955],
               [ 0.7239, -0.5734,  1.5344],
               [ 1.3664,  0.5328, -0.6955],
               [ 0.7239, -0.5734,  1.5344],
               [ 1.3664,  0.5328, -0.6955],
               [ 0.7239, -0.5734,  1.5344],
               [ 1.3664,  0.5328, -0.6955],
               [ 0.7239, -0.5734,  1.5344],
               [ 1.3664,  0.5328, -0.6955],
               [ 0.7239, -0.5734,  1.5344],
               [ 1.3664,  0.5328, -0.6955],
               [ 0.7239, -0.5734,  1.5344],
               [ 1.3664,  0.5328, -0.6955],
               [ 0.7239, -0.5734,  1.5344]]),
    x: Batch(
           c: tensor([[-0.1460,  1.6373, -0.0251, -1.8319],
                      [-0.7678, -1.0869, -1.2274, -1.3800],
                      [ 0.9530, -0.0147,  1.8584, -0.3584],
                      [-0.1460,  1.6373, -0.0251, -1.8319],
                      [-0.7678, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 228 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

314 µs ± 5.31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
